## Imports

In [5]:
import numpy as np
import pandas as pd
import re
import json
from rapidfuzz import fuzz, process

## Cleaning Function

In [6]:
def strip_clean_drop(
    dataframe,
    inplace: bool = False,
    strip_strings: bool = True,
    drop_empty: bool = True,
    run_clean_names: bool = True,
):
    """
    Clean DataFrame column names and optionally strip string cells / drop empty rows/cols.
    Returns a cleaned copy by default (set inplace=True to modify input object).
    """
    import re
    import unicodedata

    df = dataframe if inplace else dataframe.copy()

    def _clean_label(label: str) -> str:
        s = str(label)
        # normalize unicode (accents -> ascii)
        s = unicodedata.normalize("NFKD", s)
        # lower, strip
        s = s.strip().lower()
        # convert whitespace (including newlines/tabs) -> underscore
        s = re.sub(r"\s+", "_", s)
        # replace hyphens with underscore, remove percent and single quotes
        s = s.replace("-", "_").replace("%", "").replace("'", "")
        # remove common punctuation (including parentheses, colons, commas, dots)
        s = re.sub(r"[()\[\]\{\}:;.,/\\]+", "", s)
        # remove any remaining characters except alphanum and underscore
        s = re.sub(r"[^0-9a-zA-Z_]", "", s)
        # collapse multiple underscores and trim leading/trailing underscores
        s = re.sub(r"_+", "_", s).strip("_")
        return s or "col"

    # Clean column labels
    new_cols = [_clean_label(c) for c in df.columns]

    # Ensure uniqueness: append suffixes for duplicates
    counts = {}
    uniq_cols = []
    for c in new_cols:
        if c in counts:
            counts[c] += 1
            uniq_cols.append(f"{c}_{counts[c]}")
        else:
            counts[c] = 0
            uniq_cols.append(c)
    df.columns = uniq_cols

    # Drop wholly empty rows/columns if requested
    if drop_empty:
        df = df.dropna(axis=0, how="all").dropna(axis=1, how="all")

    # Convert dtypes (safe fallback if convert_string arg not supported)
    try:
        df = df.convert_dtypes(convert_string=True)
    except TypeError:
        df = df.convert_dtypes()

    # Strip string columns (applies to pd.StringDtype and object columns)
    if strip_strings:
        str_cols = df.select_dtypes(include=["string", "object"]).columns
        for col in str_cols:
            try:
                # prefer pandas string dtype for vectorized .str methods
                df[col] = df[col].astype("string").str.strip()
            except Exception:
                # best-effort: fallback to applying strip elementwise
                try:
                    df[col] = df[col].apply(lambda v: v.strip() if isinstance(v, str) else v)
                except Exception:
                    pass

    # janitor-style clean_names (if available)
    if run_clean_names:
        try:
            df = df.clean_names().convert_dtypes().reset_index(drop=True)
        except Exception:
            df = df.reset_index(drop=True)

    return df

## Loading Data

In [7]:
# read skills listing
skills_df = pd.read_excel('../data/jobsandskills-skillsfuture-skills-framework-dataset.xlsx', sheet_name=2)

skills_df = strip_clean_drop(skills_df)
skills_df.info()
skills_df.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44527 entries, 0 to 44526
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sector             44527 non-null  string
 1   track              44527 non-null  string
 2   job_role           44527 non-null  string
 3   tsc_ccs_title      44527 non-null  string
 4   tsc_ccs_type       44527 non-null  string
 5   proficiency_level  44527 non-null  string
 6   tsc_ccs_code       44527 non-null  string
dtypes: string(7)
memory usage: 2.4 MB


,sector,track,job_role,tsc_ccs_title,tsc_ccs_type,proficiency_level,tsc_ccs_code
32187,Marine and Offshore,Production Engineering,Quality Assurance / Quality Control Manager,Structural Testing,tsc,4,MAR-QUA-4006-1.1
31356,Logistics,Transportation Management and Operations,Depot Supervisor / Traffic Supervisor / Dispat...,Cargo and Receipt Inspection,tsc,3,LOG-FFO-3001-1.1


In [8]:
# read dataset
jobs_df = pd.read_csv('../data/SGJobData.csv.xz')

jobs_df = strip_clean_drop(jobs_df)
jobs_df.info()
jobs_df.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048585 entries, 0 to 1048584
Data columns (total 21 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   categories                          1044597 non-null  string 
 1   employmenttypes                     1044597 non-null  string 
 2   metadata_expirydate                 1044597 non-null  string 
 3   metadata_ispostedonbehalf           1048585 non-null  boolean
 4   metadata_jobpostid                  1044597 non-null  string 
 5   metadata_newpostingdate             1044597 non-null  string 
 6   metadata_originalpostingdate        1044597 non-null  string 
 7   metadata_repostcount                1048585 non-null  Int64  
 8   metadata_totalnumberjobapplication  1048585 non-null  Int64  
 9   metadata_totalnumberofview          1048585 non-null  Int64  
 10  minimumyearsexperience              1048585 non-null  Int64  
 11  numberofvac

,categories,employmenttypes,metadata_expirydate,metadata_ispostedonbehalf,metadata_jobpostid,metadata_newpostingdate,metadata_originalpostingdate,metadata_repostcount,metadata_totalnumberjobapplication,metadata_totalnumberofview,...,numberofvacancies,positionlevels,postedcompany_name,salary_maximum,salary_minimum,salary_type,status_id,status_jobstatus,title,average_salary
215426,"[{""id"":13,""category"":""Environment / Health""}]",Permanent,2023-08-06,False,MCF-2023-0517354,2023-07-07,2023-07-07,0,0,1,...,20,Executive,HONG YE GROUP PTE. LTD.,4000,2500,Monthly,0,Open,Cleaning Operations Executive,3250.0
904681,"[{""id"":35,""category"":""Sales / Retail""},{""id"":4...",Full Time,2024-05-04,True,MCF-2024-0488927,2024-04-04,2024-04-04,0,0,1,...,8,Executive,FK HUMAN RESOURCES PRIVATE LIMITED,4800,3150,Monthly,0,Open,COORDINATOR,3975.0


## Data Cleaning

In [9]:
# drop NA
jobs_df.isna().sum()
jobs_df = jobs_df.dropna(subset=['metadata_jobpostid'])
jobs_df.isna().sum()

categories                            0
employmenttypes                       0
metadata_expirydate                   0
metadata_ispostedonbehalf             0
metadata_jobpostid                    0
metadata_newpostingdate               0
metadata_originalpostingdate          0
metadata_repostcount                  0
metadata_totalnumberjobapplication    0
metadata_totalnumberofview            0
minimumyearsexperience                0
numberofvacancies                     0
positionlevels                        0
postedcompany_name                    0
salary_maximum                        0
salary_minimum                        0
salary_type                           0
status_id                             0
status_jobstatus                      0
title                                 0
average_salary                        0
dtype: int64

In [10]:
desc = jobs_df.describe(include='all')

In [11]:
# mask to select columns:
mask = ['metadata_jobpostid','title','postedcompany_name',
        'minimumyearsexperience','positionlevels','metadata_originalpostingdate',
        'metadata_totalnumberjobapplication','metadata_totalnumberofview',
        'numberofvacancies','categories','average_salary'
        ]


In [12]:
jobs_df_cleaned = jobs_df[mask]
jobs_df_cleaned = jobs_df_cleaned.rename(
    columns={
        'metadata_jobpostid':'job_id',
        'postedcompany_name':'company',
        'metadata_originalpostingdate':'posting_date',
        'minimumyearsexperience':'min_exp',
        'metadata_totalnumberjobapplication':'num_applications',
        'metadata_totalnumberofview':'num_views',
        'numberofvacancies':'num_vacancies'
    }
)
jobs_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1044597 entries, 0 to 1048584
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   job_id            1044597 non-null  string 
 1   title             1044597 non-null  string 
 2   company           1044597 non-null  string 
 3   min_exp           1044597 non-null  Int64  
 4   positionlevels    1044597 non-null  string 
 5   posting_date      1044597 non-null  string 
 6   num_applications  1044597 non-null  Int64  
 7   num_views         1044597 non-null  Int64  
 8   num_vacancies     1044597 non-null  Int64  
 9   categories        1044597 non-null  string 
 10  average_salary    1044597 non-null  Float64
dtypes: Float64(1), Int64(4), string(6)
memory usage: 100.6 MB


In [13]:
jobs_df_cleaned.average_salary.describe()

count       1044597.0
mean      4787.653609
std      25524.970506
min               1.0
25%            2900.0
50%            3800.0
75%            5500.0
max        12666400.0
Name: average_salary, dtype: Float64

In [14]:
def winsorize_salary_log_iqr(
    df: pd.DataFrame,
    salary_col: str = "salary",
    output_col: str = "salary_winsor"
):
    """
    Winsorize salary using log-scale IQR bounds (Method 1, winsorised).

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe
    salary_col : str
        Column containing salary values
    output_col : str
        Name of output winsorized salary column

    Returns
    -------
    pd.DataFrame
        DataFrame with winsorized salary column added
    """

    out = df.copy()

    # 1. Handle impossible values (keep rows, set to NaN)
    salary = out[salary_col].where(out[salary_col] > 0)

    # 2. Log transform
    log_salary = np.log(salary)

    # 3. Compute IQR bounds on log scale
    Q1 = log_salary.quantile(0.25)
    Q3 = log_salary.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 2.5 * IQR

    # 4. Winsorize (cap instead of drop)
    log_salary_w = log_salary.clip(
        lower=lower_bound,
        upper=upper_bound
    )

    # 5. Back-transform
    out[output_col] = np.exp(log_salary_w)

    return out

jobs_df_cleaned = winsorize_salary_log_iqr(
    jobs_df_cleaned,
    salary_col='average_salary',
    output_col='average_salary_cleaned'
)
jobs_df_cleaned['average_salary_cleaned'] = np.round(jobs_df_cleaned['average_salary_cleaned'],0)

In [15]:
def explode_categories(df, col="categories"):
    out = df.copy()

    out[col] = (
        out[col]
        .dropna()
        .map(lambda x: json.loads(x) if isinstance(x, str) else x)
    )

    out = out.explode(col, ignore_index=True)
    out["category"]    = out[col].map(lambda d: d.get("category") if isinstance(d, dict) else None)

    return out.drop(columns=[col])

jobs_df_cleaned_exploded = explode_categories(jobs_df_cleaned)
jobs_df_cleaned_exploded.sample(3)

,job_id,title,company,min_exp,positionlevels,posting_date,num_applications,num_views,num_vacancies,average_salary,average_salary_cleaned,category
552516,MCF-2023-0630768,Car Care Associate (Car Washer),LUMENS PTE. LTD.,1,Non-executive,2023-08-17,0,0,3,2000.0,2000.0,Customer Service
1514054,MCF-2024-0475974,Spa Therapist,YUAN EMPLOYMENT AGENCY,1,Non-executive,2024-04-02,0,1,10,4500.0,4500.0,Healthcare / Pharmaceutical
1028936,MCF-2023-0973951,Egglet / Waffle F&B Kiosk Supervisor,JI DAN PRIVATE LIMITED,1,Middle Management,2023-12-06,0,6,10,2250.0,2250.0,F&B


## Job Title Matching to Skills

In [16]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz

def build_title_matcher(df_skills, title_col="job_title",
                        embed_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    # canonical unique titles
    canon_titles = (
        df_skills[title_col]
        .dropna()
        .astype(str)
        .drop_duplicates()
        .tolist()
    )

    model = SentenceTransformer(embed_model_name)
    canon_emb = model.encode(canon_titles, normalize_embeddings=True, batch_size=256, show_progress_bar=True)

    return {
        "model": model,
        "canon_titles": canon_titles,
        "canon_emb": canon_emb
    }

def match_titles_embedding(
    df_dirty, dirty_col,
    matcher,
    top_k=10,
    # acceptance policy (tune on a small audit sample)
    min_sim=0.72,
    min_margin=0.05,
    use_fuzzy_rerank=True,
    min_fuzzy=85
):
    model = matcher["model"]
    canon_titles = matcher["canon_titles"]
    canon_emb = matcher["canon_emb"]

    dirty_titles = df_dirty[dirty_col].fillna("").astype(str).tolist()
    dirty_emb = model.encode(dirty_titles, normalize_embeddings=True, batch_size=256, show_progress_bar=True)

    sims = cosine_similarity(dirty_emb, canon_emb)  # shape: [n_dirty, n_canon]

    rows = []
    for i, title in enumerate(dirty_titles):
        if not title.strip():
            rows.append((None, None, None, "no_title"))
            continue

        # top-k by embedding similarity
        idx = np.argpartition(-sims[i], range(min(top_k, sims.shape[1])))[:top_k]
        idx = idx[np.argsort(-sims[i][idx])]

        cand = [(canon_titles[j], float(sims[i][j])) for j in idx]
        (t1, s1) = cand[0]
        s2 = cand[1][1] if len(cand) > 1 else 0.0
        margin = s1 - s2

        # optional fuzzy rerank inside candidates (purely algorithmic; no tokens/dicts)
        if use_fuzzy_rerank:
            fuzzy_scores = [(t, sc, fuzz.WRatio(title, t)) for (t, sc) in cand]
            # sort primarily by fuzzy, secondarily by embedding
            fuzzy_scores.sort(key=lambda x: (x[2], x[1]), reverse=True)
            t1, s1, f1 = fuzzy_scores[0]
            f2 = fuzzy_scores[1][2] if len(fuzzy_scores) > 1 else 0
        else:
            f1 = fuzz.WRatio(title, t1)
            f2 = 0

        # decision rule: require high sim AND margin, plus fuzzy sanity check if reranking
        if (s1 >= min_sim) and (margin >= min_margin) and ((not use_fuzzy_rerank) or (f1 >= min_fuzzy)):
            decision = "accept"
        elif (s1 >= (min_sim - 0.07)) or (f1 >= (min_fuzzy - 10)):
            decision = "review"
        else:
            decision = "reject"

        rows.append((t1, s1, f1, decision))

    out = df_dirty.copy()
    out["matched_job_title"] = [r[0] for r in rows]
    out["embed_sim"] = [r[1] for r in rows]
    out["fuzzy_score"] = [r[2] for r in rows]
    out["match_decision"] = [r[3] for r in rows]
    return out

def attach_skills(df_matched, df_skills, skills_title_col="job_title", skill_col="skill"):
    # Keep accepted only (recommended), then merge to skills list
    accepted = df_matched[df_matched["match_decision"] == "accept"].copy()
    enriched = accepted.merge(
        df_skills[[skills_title_col, skill_col]],
        left_on="matched_job_title",
        right_on=skills_title_col,
        how="left"
    )
    return enriched

# ------------------------
# Usage
# ------------------------
# matcher = build_title_matcher(df_skills, title_col="job_title")
# df_matched = match_titles_embedding(df_dirty, "dirty_title", matcher)
# df_with_skills = attach_skills(df_matched, df_skills)

matcher = build_title_matcher(skills_df, title_col="job_role")
df_matched = match_titles_embedding(jobs_df_cleaned, "title", matcher)
df_with_skills = attach_skills(df_matched, skills_df, skills_title_col="job_role", skill_col="tsc_ccs_title")

/Users/zacang/Documents/School/NTU SCTP DS4/Mod 1/ntu-sctp-m1-capstone/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1465.51it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 4081/4081 [03:34<00:00, 19.05it/s]


In [17]:
jobs_df_cleaned_withskills = df_with_skills.drop(columns=['embed_sim','fuzzy_score','match_decision','matched_job_title'])

In [21]:
jobs_df_cleaned_withskills = jobs_df_cleaned_withskills.rename(
    columns={
        'job_role':'jobtitle_cleaned',
        'tsc_ccs_title':'skill'
        }
)

In [22]:
jobs_df_cleaned_withskills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6263478 entries, 0 to 6263477
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   job_id                  string 
 1   title                   string 
 2   company                 string 
 3   min_exp                 Int64  
 4   positionlevels          string 
 5   posting_date            string 
 6   num_applications        Int64  
 7   num_views               Int64  
 8   num_vacancies           Int64  
 9   categories              string 
 10  average_salary          Float64
 11  average_salary_cleaned  Float64
 12  jobtitle_cleaned        string 
 13  skill                   string 
dtypes: Float64(2), Int64(4), string(8)
memory usage: 704.9 MB


## Export Data to Parquet Files

In [23]:
jobs_df_cleaned = jobs_df_cleaned.set_index('posting_date')
jobs_df_cleaned.to_parquet('../data/cleaned-sgjobdata.parquet', index=False)
jobs_df_cleaned_exploded = jobs_df_cleaned_exploded.set_index('posting_date')
jobs_df_cleaned_exploded.to_parquet('../data/cleaned-sgjobdata-exploded.parquet', index=False)
jobs_df_cleaned_withskills = jobs_df_cleaned_withskills.set_index('posting_date')
jobs_df_cleaned_withskills.to_parquet('../data/cleaned-sgjobdata-withskills.parquet', index=False)